<a href="https://colab.research.google.com/github/ArashDehghanyan/ml-practicres/blob/main/Save_and_load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup
##Installs and imports

In [1]:
! pip install pyyaml h5py   # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.version.VERSION)

2.8.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.
train_labels = train_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
test_labels = test_labels[:1000]

#Define a model
##Start by a simple Sequential model

In [4]:
def create_model():
    # Define model
    model = tf.keras.models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(10)
    ])
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

# Create a basic model instance
basic_model = create_model()

# Display the model architecture
basic_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#Save checkpoints during training

In [5]:
checkpoint_path = "training_1/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weight
checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True)

# Train model with new callback
basic_model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    # Pass callback to training
    callbacks=[checkpoint_callback]
)

Epoch 1/10
22/32 [===================>..........] - ETA: 0s - loss: 1.3224 - sparse_categorical_accuracy: 0.6406
Epoch 1: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 1.1339 - sparse_categorical_accuracy: 0.6860 - val_loss: 0.6828 - val_sparse_categorical_accuracy: 0.7860
Epoch 2/10
23/32 [====================>.........] - ETA: 0s - loss: 0.4144 - sparse_categorical_accuracy: 0.8777
Epoch 2: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4079 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.5373 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
22/32 [===================>..........] - ETA: 0s - loss: 0.2746 - sparse_categorical_accuracy: 0.9332
Epoch 3: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2813 - sparse_categorical_accuracy: 0.9310 - val_loss: 0.4706 - val_sparse_categorical_accuracy: 0.8560
Ep

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'checkpoint.ckpt.data-00000-of-00001', 'checkpoint.ckpt.index']

##Create an untrained model and evalute it on the test set

In [9]:
untrained_model = create_model()

# Evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3227 - sparse_categorical_accuracy: 0.0970 - 177ms/epoch - 6ms/step
Untrained model accuracy:  9.70%


###Load weights from checkpoint and re-evaluate the model

In [11]:
# Load weights
untrained_model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4275 - sparse_categorical_accuracy: 0.8680 - 89ms/epoch - 3ms/step
Restored model accuracy: 86.80%


##Checkpoint options

In [12]:
# Include the epoch in the filename
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback to save model's weight every 5 epochs
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

# Create a new model instance
new_model = create_model()

# Save the weights using (checkpoint_path) format
new_model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with new callback
new_model.fit(
    train_images,
    train_labels,
    epochs=50,
    verbose=1, 
    callbacks=[cp_callback],
    batch_size=batch_size,
    validation_data=(test_images, test_labels)
)

Epoch 1/50
32/32 [==============================] - 1s 13ms/step - loss: 1.1767 - sparse_categorical_accuracy: 0.6530 - val_loss: 0.7091 - val_sparse_categorical_accuracy: 0.7930
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 0.4336 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5162 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.2841 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.4731 - val_sparse_categorical_accuracy: 0.8500
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2141 - sparse_categorical_accuracy: 0.9420 - val_loss: 0.4821 - val_sparse_categorical_accuracy: 0.8430
Epoch 5/50
21/32 [==================>...........] - ETA: 0s - loss: 0.1719 - sparse_categorical_accuracy: 0.9568
Epoch 5: saving model to training_2/cp-0005.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.1716 - sparse_categorical_accuracy: 0.9610 - val_loss: 0.42

In [14]:
os.listdir(checkpoint_dir)

['cp-0035.ckpt.index',
 'cp-0015.ckpt.index',
 'checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index']

In [15]:
# Select the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
# Create a new model instance
new_untrained_model = create_model()

# Load previously saved weights
new_untrained_model.load_weights(latest)

# Re-evaluate the model
loss, acc = new_untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(acc * 100))


32/32 - 0s - loss: 0.4706 - sparse_categorical_accuracy: 0.8850 - 190ms/epoch - 6ms/step
Restored model accuracy: 88.50%
